## Chroma

In [ ]:
import chromadb
import os

In [ ]:
client = chromadb.Client()

In [ ]:
collection = client.create_collection("speaker-embeddings")

## pyannote-audio

In [ ]:
from pyannote.audio import Pipeline

In [ ]:
HF_TOKEN = os.getenv("HF_TOKEN")
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HF_TOKEN
)


In [ ]:
import torch

In [ ]:
pipeline.to(torch.device("cuda"))

In [ ]:
diarization_02 = pipeline("audio/speaker_02.wav")

In [ ]:
diarization_03 = pipeline("audio/speaker_03.wav")

In [ ]:
diarization_04 = pipeline("audio/speaker_04.wav")

In [ ]:
import json
file_names = ["speaker_02.wav", "speaker_03.wav", "speaker_04.wav"]
speaker_diarization = {fn:[] for fn in file_names}
for diar, file_name in zip([diarization_02, diarization_03, diarization_04], file_names):
    for turn, _, speaker in diar.itertracks(yield_label=True):
        # print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
        speaker_diarization[file_name].append({
            "start": turn.start,
            "end": turn.end,
            "speaker": speaker
        })
    # end for
# end for
with open("author_diarization.json", "w") as f:
    json.dump(speaker_diarization, f)
# end with